## Validação dos dados EOG

Neste notebook está incluído os seguintes passos:
- Aplicação de características;
- Criação do vetor de características;
- Normalização de dados;
- Seleção de características;
- Classificação dos dados.

In [1]:
import numpy as np

Uma característica é uma propriedade individual mensurável ou característica de um fenômeno que está sendo observado. Em nosso caso de EOG, uma característica pode ser extraída no domínio do tempo ou no domínio da frequência. As características a seguir foram retiradas do artigo *EMG Feature Extraction for Tolerance of White Gaussian Noise* \[1\].

#### Domínio do tempo

1. Willison Amplitude (WAMP)

    > $ \sum_{i=1}^{N-1}f(|x_i - x_{i+1}|) $
    
    > $ f(x) = \begin{cases} 1 & \text{if } x \gt threshold \\ 0 & \text{otherwise} \end{cases} $

2. Variance of EMG (VAR)

    > $ \frac{1}{N-1}\sum_{i=1}^{N}x_i^2 $

3. Root Mean Square (RMS)

    > $ \sqrt{\frac{1}{N}\sum_{i=1}^{N}|x_i|^2} $

4. Waveform Length (WL)
    
    > $ \sum_{i=1}^{N-1}|x_{i+1} - x_i| $

5. Zero Crossing (ZC)

    > $ \sum_{i=1}^{N}sgn(x_i) $
    
    > $ sgn(x) = \begin{cases} 1 & \text{if } x_i * x_{i+1} \leq 0 \\ 0 & \text{otherwise} \end{cases} $

#### Domínio da frequência

1. Median Frequency (FMD)

    > $ \frac{1}{2}\sum_{j=1}^{M}PSD_j $

2. Mean Frequency (FMN)

    > $\sum_{j=1}^{M}f_j PSD_j / \sum_{j=1}^{M}PSD_j$
    
    > $ f_j = j * SampleRate / 2 * M $

3. Modified Median Frequency (MMDF)

    > $ \frac{1}{2}\sum_{j=1}^{M}A_j $
    
    > $ A_j = Amplitude\ do\ espectro\ j $

4. Modified Frequency Mean (MMNF)

    > $ \sum_{j=1}^{M}f_jAj / \sum_{j=1}^{M}Aj $


\[1\] Phinyomark, Angkoon & Limsakul, Chusak & Phukpattaranont, P.. (2008). EMG Feature Extraction for Tolerance of White Gaussian Noise.
[Disponível neste link](https://www.researchgate.net/publication/263765853_EMG_Feature_Extraction_for_Tolerance_of_White_Gaussian_Noise)

**Tarefa 1**: Descrever as características de acordo com o artigo citado e outros disponíveis relacionados. O que está querendo "ser visto" em cada característica? Qual é o significado matemático de cada uma delas?

In [2]:
from math import prod
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder

# funções auxiliares
def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2


# funções de extração de características

def wamp(time, threshold):
    return np.sum(np.abs(np.diff(time)) > threshold, axis=-1)

def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape) - 1)

def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape) - 1))

def fmd(w):
    return np.sum(PSD(w), axis=-1) / 2

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2


def func_j(w):
    sample_rate = np.arange(1, w.shape[-1]+1) * 200
    return sample_rate / 2 * w.shape[-1]

def fmn(w):
    return np.sum(func_j(w) * PSD(w)) / fmd(w)*2

def mmnf(w):
    return np.sum(func_j(w) * np.abs(w),  axis=-1) / mmdf(w)*2

def wl(x):
    return np.sum(np.abs(np.diff(x, axis=-1)), axis=-1)

def zc(x):
    return np.count_nonzero(np.diff(np.sign(x), axis=-1) != 0, axis=-1)

#### Aplicando as características

É necessário implementar as características, geralmente em formato de funções ou métodos, para que seja possível aplicar tais funções aos dados de entrada e obter as características resultantes. A seguir temos a implementação das características `VAR` & `RMS` (domínio do tempo) e `FDM` & `MMDF` (domínio da frequência).

Wilston

np.sum(np.abs(d[:, i]-d[i+1, :])for i in range(N-1))]

In [3]:
from math import prod
import numpy as np

# funções auxiliares
def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2


# funções de extração de características

def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape) - 1)

def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape) - 1))

def fmd(w):
    return np.sum(PSD(w), axis=-1) / 2

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2

**Tarefa 2**: Implemente todas as características apresentadas neste tutorial em formato de funções. Sinta-se livre também para buscar e implementar características além das apresentadas, citando as fontes de tais características.


#### Vetor de características

Ao final da implementação e seleção das características, deve ser escolhida as características e então teremos um vetor com todas elas implementadas.

O vetor de características estará organizado da seguinte forma (exemplo p/ VAR, RMS, RDM e MMDF):

| ID sample | VAR1 | RMS1 | FMD1 | MMDF1 | VAR2 | RMS2 | FMD2 | MMDF2 | Classe |
|:---------:|:----:|:----:|:----:|:-----:|------|------|------|-------|:------:|
|     1     |  v1  |  v1  |  v1  |   v1  | v1   | v1   | v1   | v1    |    0   |
|     2     |  v2  |  v2  |  v2  |   v2  | v2   | v2   | v2   | v2    |    0   |
|    ...    |  ... |  ... |  ... |  ...  | ...  | ...  | ...  | ...   |   ...  |
|     N     |  vN  |  vN  |  vN  |   vN  | vN   | vN   | vN   | vN    |    7   |

#### Implementação do vetor

In [4]:
# carregando dados do "prepare.ipynb"

time_all = np.load("./dataset/all_time.npy")
freq_all = np.load("./dataset/all_freq.npy")


time = np.load("./dataset/gabi_time.npy")
freq = np.load("./dataset/gabi_freq.npy")


print("Shape original:", time_all.shape, freq_all.shape)

# aplicando características

data_var = var(time)
data_rms = rms(time)
data_wl = wl(time)
data_wamp = wamp(time, np.median(time))

data_fmd = fmd(freq)
data_mmdf = mmdf(freq)
data_fmn = fmn(freq)
data_mmnf = mmnf(freq)

print("Shape final:", time_all.shape, freq_all.shape)
print(data_wamp.shape, data_var.shape, data_rms.shape, data_fmd.shape, data_mmdf.shape, data_fmn.shape, data_mmnf.shape)

data_zc = zc(freq)


data_var.shape, data_rms.shape, data_fmd.shape, data_mmdf.shape

Shape original: (2, 28, 33, 2, 64) (2, 28, 33, 2, 33)
Shape final: (2, 28, 33, 2, 64) (2, 28, 33, 2, 33)
(28, 33, 2) (28, 33, 2) (28, 33, 2) (28, 33, 2) (28, 33, 2) (28, 33, 2) (28, 33, 2)


((28, 33, 2), (28, 33, 2), (28, 33, 2), (28, 33, 2))

# all

In [5]:
# União do vetor de características inicial
features_all = np.array([
    wamp(time_all, np.median(time_all)),
    var(time_all), 
    rms(time_all), 
    fmd(freq_all), 
    fmn(freq_all), 
    mmdf(freq_all), 
    mmnf(freq_all)
])

print("features_all", features_all.shape)

# organização das dimensões
features_all = features_all.transpose(1, 3, 2, 0, 4)
print("features_all", features_all.shape)

# criar vetor de características definitivo
feature_reshaped = features_all.reshape(features_all.shape[0],  features_all.shape[1] * features_all.shape[2],  features_all.shape[3] * features_all.shape[4])
feature_reshaped = feature_reshaped.reshape(feature_reshaped.shape[0] * feature_reshaped.shape[1], feature_reshaped.shape[2])
feature_reshaped.shape

features_all (7, 2, 28, 33, 2)
features_all (2, 33, 28, 7, 2)


(1848, 14)

# Single

In [6]:
# 1 INDIVIDUO

# União do vetor de características inicial
features = np.array([data_wamp, data_var, data_rms, data_fmd, data_mmdf, data_fmn, data_wl, data_zc])
print(features.shape)

# organização das dimensões
features = features.transpose(2, 1, 0, 3)
print(features.shape)

# # criar vetor de características definitivo
features = features.reshape(features.shape[0] * features.shape[1], features.shape[2] * features.shape[3])

features

(8, 28, 33, 2)
(33, 28, 8, 2)


array([[  3.        ,   0.        ,  23.30509552, ..., 938.00426102,
          0.        ,   1.        ],
       [  0.        ,   0.        ,  32.78020861, ..., 548.4675293 ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,  32.05401474, ..., 491.95769119,
          0.        ,   2.        ],
       ...,
       [  0.        ,   0.        ,  33.82627473, ..., 462.95471334,
         17.        ,  13.        ],
       [  0.        ,   0.        ,  21.74441401, ..., 457.10926819,
         13.        ,  13.        ],
       [  1.        ,   0.        ,  87.67349281, ..., 647.19341278,
         19.        ,  15.        ]])

*Tarefa 3*: Realização da normalização dos dados utilizando ferramentas já conhecidas

In [7]:
ss_X = StandardScaler().fit_transform(features)

*Tarefa 4*: Realização da seleção de características, utilizando ferramentas já conhecidas

Criação do vetor de *labels*

In [8]:
labels_str = ['dir', 'esq', 'cima', 'baixo', 'cima', 'baixo',
'baixo', 'esq', 'dir', 'baixo', 'dir', 'dir', 'esq', 'cima',
'baixo', 'cima', 'esq', 'dir', 'cima', 'esq', 'baixo', 'esq',
'dir', 'esq', 'cima', 'dir', 'cima', 'baixo']

# transformando para numérico
lab_dict = {'dir': 0, 'esq': 1, 'cima': 2, 'baixo': 3}
labels_num = [lab_dict[item] for item in labels_str]
print('labels_num', len(labels_num))

# criação do vetor de labels final
y = np.repeat(labels_num, int(features.shape[0] / len(labels_num)), axis=-1)
len(y)

labels_num 28


924

In [13]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_selection import VarianceThreshold, SelectKBest, RFE, f_classif
from joblib import Parallel, delayed
#from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

import itertools
from sklearn.svm import SVC

*Tarefa 5*: Realização da classificação utilizando `SVM`.

In [16]:
#Cs e gammas são listas com os valores a serem avaliados para os respectivos parâmetros.
def selecionar_melhor_svm(Cs, gammas, X_treino : np.ndarray, X_val : np.ndarray, 
                          y_treino : np.ndarray, y_val : np.ndarray, n_jobs=4, kernel='rbf'):
    
    def treinar_svm(C, gamma, X_treino, X_val, y_treino, y_val):
        svm = SVC(C=C, gamma=gamma, kernel=kernel)
        svm.fit(X_treino, y_treino)
        pred = svm.predict(X_val)
        return accuracy_score(y_val, pred)
    
    #gera todas as combinações de parametros C e gamma, de acordo com as listas de valores recebidas por parametro.
    #Na prática faz o produto cartesiano entre Cs e gammas.
    combinacoes_parametros = list(itertools.product(Cs, gammas))
    
    #Treinar modelos com todas as combinações de C e gamma
    acuracias_val = Parallel(n_jobs=n_jobs)(delayed(treinar_svm)
                                       (c, g, X_treino, X_val, y_treino, y_val) for c, g in combinacoes_parametros)       
    
    melhor_val = max(acuracias_val)
    #Encontrar a combinação que levou ao melhor resultado no conjunto de validação
    melhor_comb = combinacoes_parametros[np.argmax(acuracias_val)]   
    melhor_c = melhor_comb[0]
    melhor_gamma = melhor_comb[1]
    
    #Treinar uma SVM com todos os dados de treino e validação usando a melhor combinação de C e gamma.
    svm = SVC(C=melhor_c, gamma=melhor_gamma)
    svm.fit(np.vstack((X_treino, X_val)), [*y_treino, *y_val])

    return svm, melhor_comb, melhor_val

#Implementa a validação cruzada para avaliar o desempenho da SVM na base de dados com as instâncias X e as saídas y.
#cv_splits indica o número de partições que devem ser criadas.
#Cs é a lista com os valores C que devem ser avaliados na busca exaustiva de parametros para a SVM.
#gammas s é a lista com os valores gamma que devem ser avaliados na busca exaustiva de parametros para a SVM.
def do_cv_svm(X, y, cv_splits, Cs=[1], gammas=['scale']):

    skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=1)

    acuracias = []
    
    #pgb = tqdm(total=cv_splits, desc='Folds avaliados')
    
    for treino_idx, teste_idx in skf.split(X, y):

        X_treino = X[treino_idx]
        y_treino = y[treino_idx]

        X_teste = X[teste_idx]
        y_teste = y[teste_idx]

        X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, stratify=y_treino, test_size=0.2, random_state=1)

        ss = StandardScaler()
        ss.fit(X_treino)
        X_treino = ss.transform(X_treino)
        X_teste = ss.transform(X_teste)
        X_val = ss.transform(X_val)

        svm, _, _ = selecionar_melhor_svm(Cs, gammas, X_treino, X_val, y_treino, y_val, kernel='linear')
        pred = svm.predict(X_teste)

        acuracias.append(accuracy_score(y_teste, pred))
        
        #pgb.update(1)
        
    #pgb.close()
    
    return acuracias

In [17]:
value = ss_X.shape[1]
init = int(value / 2)
end = value + 1

for k in range(init, end):
    X_new_ss = SelectKBest(f_classif, k=k).fit_transform(ss_X, y)
    accs_svm = do_cv_svm(X_new_ss, y, 10, Cs=[1, 10, 100, 1000], gammas=['scale', 'auto', 2e-2, 2e-3, 2e-4])
    print("min: %.2f, max: %.2f, avg +- std: %.2f +- %.2f" % (min(accs_svm), max(accs_svm), np.mean(accs_svm), np.std(accs_svm)))

c:\Users\gregu\Projetos\SSVEP\r_padrao\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [1] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\gregu\Projetos\SSVEP\r_padrao\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


min: 0.32, max: 0.46, avg +- std: 0.40 +- 0.04


c:\Users\gregu\Projetos\SSVEP\r_padrao\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [1] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\gregu\Projetos\SSVEP\r_padrao\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


min: 0.32, max: 0.52, avg +- std: 0.42 +- 0.06


c:\Users\gregu\Projetos\SSVEP\r_padrao\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [1] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\gregu\Projetos\SSVEP\r_padrao\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


...

min: 0.29, max: 0.46, avg +- std: 0.38 +- 0.05


c:\Users\gregu\Projetos\SSVEP\r_padrao\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [1] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\gregu\Projetos\SSVEP\r_padrao\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


min: 0.34, max: 0.49, avg +- std: 0.41 +- 0.05


c:\Users\gregu\Projetos\SSVEP\r_padrao\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [1] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\gregu\Projetos\SSVEP\r_padrao\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


min: 0.34, max: 0.49, avg +- std: 0.41 +- 0.05


maior acurácia encontrada foi de 45%